# Intro

Perform Sentiment analysis using DistilBERT on Yelp reviews - Full and apply the EDA augmentation. Report the performance metrics for before and after augmentation? show examples of data augmentaion?

* data https://course.fast.ai/datasets
* papers: https://arxiv.org/pdf/1910.01108.pdf & https://arxiv.org/pdf/1901.11196.pdf
* Hint on dataset Downsample the dataset according to your computing resources.

In [1]:
!pip install transformers==3.5.1 

In [2]:
!pip install ohmeow-blurr==0.0.18

In [22]:
!pip install spacy

In [23]:
!pip install -U nlp_profiler

  Using cached nlp_profiler-0.0.2-py2.py3-none-any.whl (39 kB)
  Using cached swifter-1.0.7.tar.gz (633 kB)
  Using cached language_tool_python-2.4.7-py3-none-any.whl (30 kB)
  Using cached emoji-0.6.0.tar.gz (51 kB)
ERROR: Could not find a version that satisfies the requirement en-core-web-sm (from nlp_profiler) (from versions: none)
ERROR: No matching distribution found for en-core-web-sm (from nlp_profiler)


In [1]:
import torch
from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

# yelp

In [2]:
path = untar_data(URLs.YELP_REVIEWS)
# path = untar_data(URLs.YELP_REVIEWS_POLARITY)

In [3]:
path.ls()

(#3) [Path('/home/tyoc213/.fastai/data/yelp_review_full_csv/train.csv'),Path('/home/tyoc213/.fastai/data/yelp_review_full_csv/readme.txt'),Path('/home/tyoc213/.fastai/data/yelp_review_full_csv/test.csv')]

In [4]:
%%time
pd_train = pd.read_csv(path/'train.csv', header=0, names=['stars', 'review'])
pd_test = pd.read_csv(path/'test.csv', header=0, names=['stars', 'review'])

CPU times: user 3.33 s, sys: 276 ms, total: 3.6 s
Wall time: 3.6 s


In [5]:
pd_train.head()

,stars,review
0,2,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I..."
1,4,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."
2,4,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
3,1,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you c..."
4,5,Top notch doctor in a top notch practice. Can't say I am surprised when I was referred to him by another doctor who I think is wonderful and because he went to one of the best medical schools in the country. \nIt is really easy to get an appointment. There is minimal wait to be seen and his bedside manner is great.


In [6]:
pd_test.head()

,stars,review
0,1,"Don't waste your time. We had two different people come to our house to give us estimates for a deck (one of them the OWNER). Both times, we never heard from them. Not a call, not the estimate, nothing."
1,1,"All I can say is the worst! We were the only 2 people in the place for lunch, the place was freezing and loaded with kids toys! 2 bicycles, a scooter, and an electronic keyboard graced the dining room. A fish tank with filthy, slimy fingerprints smeared all over it is there for your enjoyment.\n\nOur food came... no water to drink, no tea, medium temperature food. Of course its cold, just like the room, I never took my jacket off! The plates are too small, you food spills over onto some semi-clean tables as you sit in your completely worn out booth seat. The fried noodles were out of a box..."
2,1,"I have been to this restaurant twice and was disappointed both times. I won't go back. The first time we were there almost 3 hours. It took forever to order and then forever for our food to come and the place was empty. When I complained the manager was very rude and tried to blame us for taking to long to order. It made no sense, how could we order when the waitress wasn't coming to the table? After arguing with me he ended up taking $6 off of our $200+ bill. Ridiculous. If it were up to me I would have never returned. Unfortunately my family decided to go here again tonight. Again it too..."
3,1,Food was NOT GOOD at all! My husband & I ate here a couple weeks ago for the first time. I ordered a salad & basil pesto cream pasta & my husband ordered the spinach & feta pasta. The salad was just a huge plate of spring mix (nothing else in it) with WAY to much vinegar dressing. My lettuce was drowning in the vinegar. My pesto pasta had no flavor (did not taste like a cream sauce to me) & the pesto was so runny/watery & way too much sauce not enough noodles. My husband's pasta had even less flavor than mine. We ate about a quarter of the food & couldn't even finish it. We took it home & ...
4,3,"This is a tiny Starbucks and it locations like this (although cute) makes you wonder if your really meant to hang out or just grab your coffee and leave. Leaving is always a good idea at this location anyway since you have a nice fountain in the back with benches and it is a central part of the Waterfront Shopping. \n\nStarbuck isn't my favorite coffee chain by any means. Is it just me or do all Starbuck coffees taste a little burnt and bitter? No matter how trendy, cool and upscale their establishments are I can't get around the yicky tasting bitterness of Staryucks regular coffees. Talk ..."


In [7]:
dir(HF_TASKS_AUTO)

['CausalLM',
 'LMHead',
 'MaskedLM',
 'MultipleChoice',
 'NextSentencePrediction',
 'PreTraining',
 'QuestionAnswering',
 'Seq2SeqLM',
 'SequenceClassification',
 'TokenClassification',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [8]:
%%time

task = HF_TASKS_AUTO.SequenceClassification

pretrained_model_name = "distilbert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,  task=task)

CPU times: user 1.8 s, sys: 140 ms, total: 1.94 s
Wall time: 2.98 s


## merge frames

In [9]:
%%time

pd_train['is_valid'] = False
pd_test['is_valid'] = True
df = pd.concat([pd_train, pd_test])

CPU times: user 32.4 ms, sys: 7.29 ms, total: 39.7 ms
Wall time: 39.1 ms


In [10]:
df.head()

,stars,review,is_valid
0,2,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I...",False
1,4,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life.",False
2,4,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!,False
3,1,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you c...",False
4,5,Top notch doctor in a top notch practice. Can't say I am surprised when I was referred to him by another doctor who I think is wonderful and because he went to one of the best medical schools in the country. \nIt is really easy to get an appointment. There is minimal wait to be seen and his bedside manner is great.,False


In [11]:
df.tail()

,stars,review,is_valid
49994,1,"Just wanted to write a review to chip in with what the others said. I would not recommend going. If you have little kids (under 8 maybe), they might like it but unless you have a lot of money to waste it probably wouldn't be worth finding out. A lot of the attractions are not very accurate. There's one where you strike a pose and you summon a superhero based on your pose. 8 out of 10 people would change their pose and summon the same superhero. Also, there's nowhere to sit. You wait in line after line and nowhere to sit. By the end, we were excited for the sitting ride. That actually was ...",True
49995,5,Great ambience. Great drinks. Great food. I love this place!,True
49996,4,"I have been to the other Monks locations so I was excited when I heard they were coming Sun Prairie. I was not disappointed. We each orded something different and shared. I really liked my chicken sandwich and the fish tacos were great. My husband had a breakfast burger that while not something I would normally order, was pretty good.\n\nThere really aren't many places in Sun Prairie to eat besides fast food and I can see Monks being one of our favorites. I took away a star because the dining room is packed with tables, when full it is a tight squeeze to get around.",True
49997,2,Don't go here. I know you might want to try it but these good reviews are people who sadly have no taste buds. I got fries a burger and the spicy chicken. The spicy chicken which I am almost positive they drive down the street got it from McDonald's re wrapped it and gave it to me for 50 cents more.. Only McDonald's chicken is better quality.. That's right McDonald's is better which is the only time I have or will ever say that. Their burger was flavorless and had the same consistency as burgers you find in the frozen dinner isle at Walmart. They have a lot of bells and whistles. Like wi...,True
49998,1,"Buffet was recently open after renovation so my husband and I are thinking it should be pretty good....wrong. We arrived at 8:30 am for what we thought was a Saturday brunch offering champs, nope. Price is $21.95 but on Monday - Friday is $18.95...same food so why the increase? I can go to the Fiesta in Henderson with champs and better food for $8.99 Sat/Sun but we wanted to try something we thought was going to be an upgrade....lol what a joke. \n\n\nFood: everything I tried looked and tasted like food was leftover from the day before and reheated. \n\nShrimps were all water logged, the c...",True


In [33]:
%%time
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), HF_TokenCategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('review'), get_y=ColReader('stars'), 
                   splitter=ColSplitter(col='is_valid'))



CPU times: user 0 ns, sys: 858 µs, total: 858 µs
Wall time: 865 µs


In [34]:
device=torch.device('cuda')

In [ ]:
%%time
dls = dblock.dataloaders(df, bs=1,device=device)
dls.device

In [ ]:
%%time
model = HF_BaseModelWrapper(hf_model)
next(model.parameters()).device

In [ ]:
dls.device, dls.vocab

In [ ]:
dls.one_batch()

In [ ]:
#slow
learn = Learner(dls, 
                model,
#                 opt_func=partial(Adam, decouple_wd=True),
#                 loss_func=CrossEntropyLossFlat(),
#                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

In [ ]:
learn.fit(1,lr=0.01)

In [ ]:
# learn.fit_one_cycle(3, lr_max=1e-3)

In [26]:
print(learn.token_classification_report)

AttributeError: 'Learner' object has no attribute 'token_classification_report'